In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import transforms, datasets, models
from torch.cuda.amp import GradScaler, autocast
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm
import numpy as np
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix,
                            roc_auc_score, roc_curve, f1_score, precision_score, 
                            recall_score)
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# VGG16 model fonksiyonları için bakınız: Cell 15 (vgg16_model.py)


In [ ]:
def get_data_loaders(data_dir, batch_size=32, img_size=224, use_weighted_sampler=False, num_workers=0):
    train_transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    val_transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    train_dataset = datasets.ImageFolder(
        root=os.path.join(data_dir, 'train'),
        transform=train_transform
    )
    
    val_dataset = datasets.ImageFolder(
        root=os.path.join(data_dir, 'val'),
        transform=val_transform
    )
    
    train_sampler = None
    class_weights = None
    
    if use_weighted_sampler:
        class_counts = {}
        for idx, (path, class_idx) in enumerate(train_dataset.samples):
            class_counts[class_idx] = class_counts.get(class_idx, 0) + 1
        
        total_samples = sum(class_counts.values())
        num_classes = len(class_counts)
        class_weights = [total_samples / (num_classes * count) for count in class_counts.values()]
        
        sample_weights = [class_weights[class_idx] for _, class_idx in train_dataset.samples]
        train_sampler = WeightedRandomSampler(
            weights=sample_weights,
            num_samples=len(sample_weights),
            replacement=True
        )
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=(train_sampler is None),
        sampler=train_sampler,
        num_workers=num_workers,
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )
    
    if not class_weights:
        class_counts = {}
        for _, class_idx in train_dataset.samples:
            class_counts[class_idx] = class_counts.get(class_idx, 0) + 1
        total_samples = sum(class_counts.values())
        num_classes = len(class_counts)
        class_weights = [total_samples / (num_classes * count) for count in class_counts.values()]
    
    return train_loader, val_loader, train_dataset.classes, class_weights


In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device, scaler=None):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    progress_bar = tqdm(train_loader, desc='Training')
    for images, labels in progress_bar:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        if scaler is not None:
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        
        progress_bar.set_postfix({'loss': loss.item()})
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = accuracy_score(all_labels, all_preds)
    
    return epoch_loss, epoch_acc


In [ ]:
def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        progress_bar = tqdm(val_loader, desc='Validation')
        for images, labels in progress_bar:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            
            probs = torch.softmax(outputs, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            
            progress_bar.set_postfix({'loss': loss.item()})
    
    epoch_loss = running_loss / len(val_loader)
    epoch_acc = accuracy_score(all_labels, all_preds)
    
    all_probs = np.array(all_probs)
    
    if len(np.unique(all_labels)) == 2:
        auc_roc = roc_auc_score(all_labels, all_probs[:, 1])
    else:
        auc_roc = roc_auc_score(all_labels, all_probs, multi_class='ovr', average='weighted')
    
    f1 = f1_score(all_labels, all_preds, average='weighted')
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    
    return epoch_loss, epoch_acc, all_preds, all_labels, all_probs, {
        'auc_roc': auc_roc,
        'f1_score': f1,
        'precision': precision,
        'recall': recall
    }


In [ ]:
def plot_training_history(history, save_path='training_history.png'):
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    epochs = range(1, len(history['train_loss']) + 1)
    
    axes[0, 0].plot(epochs, history['train_loss'], label='Eğitim Loss', marker='o')
    axes[0, 0].plot(epochs, history['val_loss'], label='Doğrulama Loss', marker='s')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('Eğitim ve Doğrulama Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    axes[0, 1].plot(epochs, history['train_acc'], label='Eğitim Accuracy', marker='o')
    axes[0, 1].plot(epochs, history['val_acc'], label='Doğrulama Accuracy', marker='s')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].set_title('Eğitim ve Doğrulama Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    if 'val_auc_roc' in history:
        axes[1, 0].plot(epochs, history['val_auc_roc'], label='Doğrulama AUC-ROC', 
                       marker='s', color='green')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('AUC-ROC')
        axes[1, 0].set_title('Doğrulama AUC-ROC')
        axes[1, 0].legend()
        axes[1, 0].grid(True)
    
    if 'val_f1_score' in history:
        axes[1, 1].plot(epochs, history['val_f1_score'], label='Validation F1-Score', 
                       marker='s', color='purple')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('F1-Score')
        axes[1, 1].set_title('Validation F1-Score')
        axes[1, 1].legend()
        axes[1, 1].grid(True)
    else:
        axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"Eğitim grafiği kaydedildi: {save_path}")


In [ ]:
def plot_confusion_matrix(y_true, y_pred, class_names, save_path='confusion_matrix.png'):
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'label': 'Örnek Sayısı'})
    plt.title('Karışıklık Matrisi (Confusion Matrix)')
    plt.ylabel('Gerçek Etiket (True Label)')
    plt.xlabel('Tahmin Edilen Etiket (Predicted Label)')
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"Karışıklık matrisi kaydedildi: {save_path}")


In [ ]:
def plot_roc_curve(y_true, y_probs, class_names, save_path='roc_curve.png'):
    n_classes = len(class_names)
    
    plt.figure(figsize=(10, 8))
    
    if n_classes == 2:
        fpr, tpr, _ = roc_curve(y_true, y_probs[:, 1])
        auc_score = roc_auc_score(y_true, y_probs[:, 1])
        plt.plot(fpr, tpr, label=f'{class_names[1]} (AUC = {auc_score:.3f})', linewidth=2)
    else:
        for i in range(n_classes):
            y_true_binary = (y_true == i).astype(int)
            fpr, tpr, _ = roc_curve(y_true_binary, y_probs[:, i])
            auc_score = roc_auc_score(y_true_binary, y_probs[:, i])
            plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {auc_score:.3f})', linewidth=2)
    
    plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier', linewidth=1)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity)')
    plt.title('ROC Eğrisi (ROC Curve)')
    plt.legend(loc='lower right')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"ROC eğrisi kaydedildi: {save_path}")


In [ ]:
PROJECT_ROOT = os.getcwd()
DATA_DIR = os.path.join(PROJECT_ROOT, 'data')
BATCH_SIZE = 48
NUM_EPOCHS = 10
LEARNING_RATE = 0.0001
IMG_SIZE = 224
NUM_CLASSES = None
SAVE_DIR = os.path.join(PROJECT_ROOT, 'checkpoints')

USE_TRANSFER_LEARNING = True
MODEL_NAME = 'vgg16'
FREEZE_BACKBONE = True
AUTO_UNFREEZE = True
UNFREEZE_EPOCH = 5
PRETRAINED = True

USE_WEIGHTED_SAMPLER = True
USE_WEIGHTED_LOSS = True

USE_SCHEDULER = True
SCHEDULER_TYPE = 'cosine'

os.makedirs(SAVE_DIR, exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("="*70)
print(f"Cihaz kullanımı: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print("="*70)


In [ ]:
if AUTO_UNFREEZE and UNFREEZE_EPOCH > NUM_EPOCHS:
    UNFREEZE_EPOCH = NUM_EPOCHS

print("\nVeriler yükleniyor...")
train_loader, val_loader, class_names, class_weights = get_data_loaders(
    DATA_DIR,
    batch_size=BATCH_SIZE,
    img_size=IMG_SIZE,
    use_weighted_sampler=USE_WEIGHTED_SAMPLER,
    num_workers=0
)
print(f"\n✓ Veriler başarıyla yüklendi!")
print(f"  Sınıflar: {class_names}")
print(f"  Eğitim görüntü sayısı: {len(train_loader.dataset)}")
print(f"  Doğrulama görüntü sayısı: {len(val_loader.dataset)}")

detected_num_classes = len(class_names)
if NUM_CLASSES is None:
    NUM_CLASSES = detected_num_classes
elif NUM_CLASSES != detected_num_classes:
    print(f"\n⚠️  NUM_CLASSES değeri ({NUM_CLASSES}) ile verilerden okunan sınıf sayısı "
          f"({detected_num_classes}) farklı. Verilerdeki değer kullanılacak.")
    NUM_CLASSES = detected_num_classes
print(f"  Toplam sınıf: {NUM_CLASSES}")


In [ ]:
print(f"\nVGG16 Model oluşturuluyor...")
print(f"  Model tipi: Transfer Learning (VGG16)")
model = create_model(
    num_classes=NUM_CLASSES,
    input_channels=3,
    pretrained=PRETRAINED,
    model_type='transfer',
    model_name='vgg16',
    freeze_backbone=FREEZE_BACKBONE
)

model = model.to(device)
backbone_unfrozen = not FREEZE_BACKBONE
if FREEZE_BACKBONE:
    print("  Backbone katmanları başlangıçta donduruldu.")

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"  Toplam parametreler: {total_params:,}")
print(f"  Eğitilebilir parametreler: {trainable_params:,}")


In [ ]:
if USE_WEIGHTED_LOSS and class_weights:
    weights = torch.FloatTensor(class_weights).to(device)
    criterion = nn.CrossEntropyLoss(weight=weights)
    print(f"\n✓ Weighted Loss etkinleştirildi")
    print(f"  Ağırlıklar: {dict(zip(class_names, class_weights))}")
else:
    criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

use_amp = torch.cuda.is_available() and hasattr(torch.cuda, 'amp')
scaler = GradScaler() if use_amp else None
if use_amp:
    print(f"\n✓ Mixed Precision Training etkinleştirildi (hızlandırma için)")

scheduler = None
if USE_SCHEDULER:
    if SCHEDULER_TYPE == 'plateau':
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=5
        )
    elif SCHEDULER_TYPE == 'cosine':
        scheduler = optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=NUM_EPOCHS, eta_min=1e-6
        )
    elif SCHEDULER_TYPE == 'step':
        scheduler = optim.lr_scheduler.StepLR(
            optimizer, step_size=15, gamma=0.5
        )
    print(f"\n✓ Learning Rate Scheduler etkinleştirildi ({SCHEDULER_TYPE})")


In [ ]:
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'val_auc_roc': [],
    'val_f1_score': [],
    'val_precision': [],
    'val_recall': []
}

best_val_acc = 0.0
best_val_auc = 0.0

print("\n" + "="*70)
print("VGG16 Transfer Learning Eğitimi başlatılıyor...")
print("="*70)


In [ ]:
for epoch in range(NUM_EPOCHS):
    print(f"\n{'='*70}")
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    print(f"{'='*70}")
    
    if FREEZE_BACKBONE and AUTO_UNFREEZE and not backbone_unfrozen and (epoch + 1) >= UNFREEZE_EPOCH:
        print(f"--> Epoch {epoch+1}: Backbone katmanları serbest bırakılıyor.")
        for param in model.parameters():
            param.requires_grad = True
        backbone_unfrozen = True
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
        if scheduler:
            if SCHEDULER_TYPE == 'cosine':
                scheduler = optim.lr_scheduler.CosineAnnealingLR(
                    optimizer, T_max=NUM_EPOCHS - epoch, eta_min=1e-6
                )
    
    train_loss, train_acc = train_epoch(
        model, train_loader, criterion, optimizer, device, scaler
    )
    
    val_loss, val_acc, val_preds, val_labels, val_probs, metrics = validate(
        model, val_loader, criterion, device
    )
    
    if scheduler:
        if SCHEDULER_TYPE == 'plateau':
            scheduler.step(val_loss)
        else:
            scheduler.step()
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['val_auc_roc'].append(metrics['auc_roc'])
    history['val_f1_score'].append(metrics['f1_score'])
    history['val_precision'].append(metrics['precision'])
    history['val_recall'].append(metrics['recall'])
    
    current_lr = optimizer.param_groups[0]['lr']
    print(f"\nEğitim Loss: {train_loss:.4f}, Eğitim Acc: {train_acc:.4f}")
    print(f"Doğrulama Loss: {val_loss:.4f}, Doğrulama Acc: {val_acc:.4f}")
    print(f"Doğrulama AUC-ROC: {metrics['auc_roc']:.4f}, Doğrulama F1-Score: {metrics['f1_score']:.4f}")
    print(f"Doğrulama Precision: {metrics['precision']:.4f}, Doğrulama Recall: {metrics['recall']:.4f}")
    print(f"Öğrenme Oranı: {current_lr:.6f}")
    
    save_model = False
    if metrics['auc_roc'] > best_val_auc:
        best_val_auc = metrics['auc_roc']
        save_model = True
        print(f"✓ AUC-ROC iyileşti! (AUC: {best_val_auc:.4f})")
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        save_model = True
        print(f"✓ Accuracy iyileşti! (Acc: {best_val_acc:.4f})")
    
    if save_model:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'val_auc_roc': metrics['auc_roc'],
            'class_names': class_names,
            'model_type': 'transfer',
            'model_name': MODEL_NAME
        }, os.path.join(SAVE_DIR, 'best_vgg16_model.pth'))


In [ ]:
print("\n" + "="*70)
print("Final Değerlendirme")
print("="*70)

print("\nSınıflandırma Raporu:")
print(classification_report(val_labels, val_preds, target_names=class_names))

print("\nGrafikler çiziliyor...")
plot_training_history(history, save_path=os.path.join(SAVE_DIR, 'training_history.png'))
plot_confusion_matrix(val_labels, val_preds, class_names, 
                     save_path=os.path.join(SAVE_DIR, 'confusion_matrix.png'))
plot_roc_curve(val_labels, val_probs, class_names, 
              save_path=os.path.join(SAVE_DIR, 'roc_curve.png'))

print("\n" + "="*70)
print("Final Sonuç Özeti:")
print("="*70)
print(f"En iyi Accuracy: {best_val_acc:.4f}")
print(f"En iyi AUC-ROC: {best_val_auc:.4f}")
print(f"Final F1-Score: {metrics['f1_score']:.4f}")
print(f"Final Precision: {metrics['precision']:.4f}")
print(f"Final Recall: {metrics['recall']:.4f}")
print("="*70)
print(f"\n✓ VGG16 Transfer Learning eğitimi başarıyla tamamlandı!")
print(f"  Model kaydedildi: {os.path.join(SAVE_DIR, 'best_vgg16_model.pth')}")


![alt text](checkpoints/confusion_matrix.png)


![alt text](checkpoints/training_history.png)


![alt text](<Ekran görüntüsü 2025-12-30 135346.png>)
